### Data Preprocessing

In [3]:
import pandas as pd

In [62]:
raw_data = pd.read_csv('data/steam-200k.csv', names=['user_id', 'game_name', 'behavior', 'hours','Description'])#.sample(10)
len(raw_data.game_name.unique())

5155

In [63]:
len(raw_data[raw_data['behavior']=='purchase'])

129511

In [64]:
len(raw_data[raw_data['behavior']=='play'])

70489

In [65]:
raw_data.columns

Index(['user_id', 'game_name', 'behavior', 'hours', 'Description'], dtype='object')

In [66]:
raw_data.shape

(200000, 5)

In [67]:
raw_data[raw_data['behavior']=="play"]['game_name']

1         The Elder Scrolls V Skyrim
3                          Fallout 4
5                              Spore
7                  Fallout New Vegas
9                      Left 4 Dead 2
                     ...            
199991                  Fallen Earth
199993                   Magic Duels
199995                   Titan Souls
199997    Grand Theft Auto Vice City
199999                          RUSH
Name: game_name, Length: 70489, dtype: object

In [68]:
#played but not purchased
played_df = raw_data[(raw_data["behavior"] == "play") & (raw_data["hours"] > 0) & (raw_data["hours"].notnull())]
played_df["game_name"].nunique()


3600

#### Adding game description for content based game

In [69]:
import requests
import json
import re

# raw_data = pd.read_csv('data/steam-200k.csv', names=['user_id', 'game_name', 'behavior', 'hours','Description'])

api_key = '612a9eda748984d9588049fdb563b6bb1776d6d0'
def descriptionFetcher(game_name):
    url = f'https://www.giantbomb.com/api/search/?api_key={api_key}&format=json&query={game_name}&resources=game'
    response = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        content = json.loads(response.text)

        descriptions = ""

        for result in content["results"]:
            if result["description"] != None:
                descriptions += result["description"]
        
        #remove html tags
        clean = re.compile('<.*?>')
        descriptions = re.sub(clean, '', descriptions)
        return descriptions
    else:
        print(response)

In [71]:
unique_games = raw_data["game_name"].unique()
# raw_data['Description'] = raw_data['game_name'].apply(descriptionFetcher)
game_info_dict = {}
for game_name in unique_games:
    try:
        game_info = descriptionFetcher(game_name)
        game_info_dict[game_name] = game_info
    except:
        print(game_name)
raw_data["Description[giantbomb]"] = raw_data["game_name"].map(game_info_dict)

<Response [502]>
<Response [502]>
The Settlers 7 Paths to a Kingdom The Two Kings DLC #4
<Response [502]>


In [ ]:
raw_data.to_csv('data-Description-Giantbomb.csv')

,user_id,game_name,behavior,hours,Description
161810,161623464,War Inc. Battlezone,purchase,1.0,System RequirementsMinimumOS: Windows XP SP3 o...
60107,66255019,Fallout New Vegas Dead Money,purchase,1.0,StoryThe Vit-O-Matic allows players to place t...
187142,126887624,Fallout New Vegas,play,0.5,StoryThe Vit-O-Matic allows players to place t...
70687,43955374,Terraria,play,144.0,OverviewTerraria is a 2D indie fantasy action-...
156199,205285563,Age of Empires III Complete Collection,play,3.1,"OverviewReleased in 2005, Age of Empires III (..."
166219,218317467,Saints Row 2,purchase,1.0,OverviewPlayers can cause large amounts of cha...
115543,88775181,Escape,play,0.5,Escape is a horror game developed and publishe...
164560,176354321,War Thunder,purchase,1.0,War Thunder is an MMO-style combat game about ...
91090,55334302,Total War SHOGUN 2,play,152.0,OverviewTotal War: Shogun 2 returns to Japan's...
80852,119619506,Infestation Survivor Stories,play,6.5,OverviewInfestation: Survivor Stories (formerl...


In [14]:
raw_data.to_csv('data.csv')

Wikipedia

In [48]:
import wikipedia
p=wikipedia.page('Revenge of the Titans')
p.content

'Revenge of the Titans is a tower defense/real-time strategy video game developed and published by Puppy Games. It was released for one of the five games included in the second Humble Indie Bundle in 2010. As a result of the success of the Humble Indie Bundle sale the source code of Revenge of the Titans was released. On March 16, 2011, the game was released on Steam.The game received positive reviews from critics, who commended its art style, gameplay, and writing.\n\n\n== Gameplay ==\n\nRevenge of the Titans plays like a traditional tower defense game but with the caveat that enemies ignore or attempt to avoid the towers that you place, increasing the difficulty. The goal for each level is to prevent the command center from being destroyed by aliens by protecting it with turrets and other defenses. The player has access to extensive research trees that they can use to develop different types of defenses against enemies.In addition to the normal alien enemies, there are also large ali

In [49]:
import wikipedia

unique_games = raw_data["game_name"].unique()

# Define a function to get the game's wikipedia page and extract its content
def get_game_info(game_name):
    try:
        page = wikipedia.page(game_name)
        clean = re.compile('<.*?>')
        description = re.sub(clean, '', page.content)
        return description
    except:
        return ""

# Apply the function to each row in the dataframe and store the results in a new column
# raw_data["Description"] = raw_data["game_name"].apply(get_game_info)
# Loop through the unique game names and store the corresponding information in a dictionary
game_info_dict = {}
for game_name in unique_games:
    game_info = get_game_info(game_name)
    game_info_dict[game_name] = game_info

raw_data["Description"] = raw_data["game_name"].map(game_info_dict)

e:\projects\steam-video-game-recommendation\venv\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file e:\projects\steam-video-game-recommendation\venv\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [55]:
get_game_info("Alice Madness Returns")

''

In [30]:
len(raw_data[raw_data['behavior']=='purchase'])

129511

In [73]:
raw_data.to_csv('data-giantbomb.csv')